In [1]:
# подразумевается что запускаем данный ноутбук с библиотеками имеющимися на google colab. В случае запуска локально,
# пожалуйста установите необходимые библиотеки такие как numpy, pandas, torch и тд. 

!pip install sentence_transformers
!pip install termcolor
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from termcolor import colored
import torch

df = pd.read_csv('test_data.csv').copy()


# нужно:
# 1) извлекать реплики, где менеджер поздоровался
# 2) извлекать реплики, где менеджер представил себя
# 3) извлекать имя менеджера
# 4) извлекать название компании (возможно в репликах клиента тоже)
# 5) извлекать реплики где менеджер попрощался
# 6) проверять требование к менеджеру, обязательно надо поздороваться и попрощаться


In [3]:
df # посмотрим на базу данных

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


In [4]:
# загрузим большую модель для более качественного построения эмбеддингов

bert = SentenceTransformer('sberbank-ai/sbert_large_mt_nlu_ru') 

In [5]:
# поскольку проверять большинство пунктов надо именно у менеджера, поэтому именно 
# реплики менеджера мы проведем через модель и получим эмбеддинги, тем самым сэкономив время вычислений

manager_encondings = bert.encode(df['text'][df['role'] == 'manager'].values)

In [6]:
# для того, чтобы найти предложения, в которых менеджер приветствуют клиента,
# постараемся подобрать синонимичные фразы, далее их закодируем и с помощью косинусной меры схожести
# попытаемся найти нужные предложения

greeting_encoding = bert.encode(['Здравствуйте','Привет', 'Добрый день', 'Доброе утро'])

In [7]:
# для удобства напишем функцию, которая по эмбеддингам определяет нужные индексы реплик в базе данных
# гиперпараметрами в данном случае выступят порог и аггрериующая статистика. 

def indices_of(needed_embeddings, manager_embeddings, threshold, aggr_statistics='mean'):
    indices = []
    stat = 0
    similarities = cosine_similarity(needed_embeddings, manager_embeddings)
    for i in range(len(similarities[0])):
        if aggr_statistics=='mean':
            stat = similarities[:, i].mean()
        elif aggr_statistics =='max':
            stat = similarities[:, i].max()
        if  stat > threshold:
            indices.append(df['text'][df['role'] == 'manager'].index[i])
    return indices


In [8]:
# вызовем написанную функцию и проверим её работоспособность

greeting_indices = indices_of(greeting_encoding, manager_encondings, 0.6)
df['text'][greeting_indices]

1             Алло здравствуйте
110           Алло здравствуйте
166           Алло здравствуйте
250    Алло дмитрий добрый день
Name: text, dtype: object

In [9]:
# по заданию нам также надо находить реплики где менеджер прощается с клиентом
# аналогично приветствию проделаем шаги 

bye_encoding = bert.encode(['До свидания', 'До встречи','Пока', 'Всего хорошего'])
bye_indices = indices_of(bye_encoding, manager_encondings, 0.47)
df['text'][bye_indices]

108                           Всего хорошего до свидания
163                                          До свидания
300    Угу все хорошо да понедельника тогда всего доб...
335    Во вторник все ну с вами да тогда до вторника ...
479                       Ну до свидания хорошего вечера
Name: text, dtype: object

In [10]:
# Пункт с нахождением реплик где менеджер представился также можно сделать этим методом с помощью 
# синонимичных выражений. Однако за счет того, что в предложениях, где менеджер представился
# присутствует много различных слов помимо нужных, метрика среднего по синонимам показывает себя хуже
# и было принято решение в данном случае использовать максимум по синонимам. Так удалось найти параметры,
# которые позволили дать более точный ответ. 

name_encoding = bert.encode(['Меня зовут', 'Мое имя', 'Да это мое имя'])
name_indices = indices_of(name_encoding, manager_encondings, 0.42, 'max')
df['text'][name_indices]

3      Меня зовут ангелина компания диджитал бизнес з...
111    Меня зовут ангелина компания диджитал бизнес з...
167    Меня зовут ангелина компания диджитал бизнес з...
251    Добрый меня максим зовут компания китобизнес у...
338                                     Да это анастасия
Name: text, dtype: object

In [11]:
# для извлечения имен и названий компаний здесь нужно использовать Named Entity Recognition.
# в ходе работы были рассмотрены различные модели (spacy/ru_core_news_sm,
# spacy/ru_core_news_lg,  Babelscape/wikineural-multilingual-ner) однако данные модели не показали себя хорошо
# в данной задаче. И в целом, NER на русском решается менее качественно, чем на английском,
# однако модель, которую нашел ниже, показывает лучше результаты чем предыдущие, поэтому продолжим 
# попытки именно с ней

class Ner_Extractor:
    """
    Labeling each token in sentence as named entity

    :param model_checkpoint: name or path to model 
    :type model_checkpoint: string
    """
    
    def __init__(self, model_checkpoint: str):
        self.token_pred_pipeline = pipeline("token-classification", 
                                            model=model_checkpoint, 
                                            aggregation_strategy="average")
    
    @staticmethod
    def text_color(txt, txt_c="blue", txt_hglt="on_yellow"):
        """
        Coloring part of text 
        
        :param txt: part of text from sentence 
        :type txt: string
        :param txt_c: text color  
        :type txt_c: string        
        :param txt_hglt: color of text highlighting  
        :type txt_hglt: string
        :return: string with color labeling
        :rtype: string
        """
        return colored(txt, txt_c, txt_hglt)
    
    @staticmethod
    def concat_entities(ner_result):
        """
        Concatenation entities from model output on grouped entities
        
        :param ner_result: output from model pipeline 
        :type ner_result: list
        :return: list of grouped entities with start - end position in text
        :rtype: list
        """
        entities = []
        prev_entity = None
        prev_end = 0
        for i in range(len(ner_result)):
            
            if (ner_result[i]["entity_group"] == prev_entity) &\
               (ner_result[i]["start"] == prev_end):
                
                entities[i-1][2] = ner_result[i]["end"]
                prev_entity = ner_result[i]["entity_group"]
                prev_end = ner_result[i]["end"]
            else:
                entities.append([ner_result[i]["entity_group"], 
                                 ner_result[i]["start"], 
                                 ner_result[i]["end"]])
                prev_entity = ner_result[i]["entity_group"]
                prev_end = ner_result[i]["end"]
        
        return entities
    
    def colored_text(self, text: str, entities: list):
        """
        Highlighting in the text named entities
        
        :param text: sentence or a part of corpus
        :type text: string
        :param entities: concated entities on groups with start - end position in text
        :type entities: list
        :return: Highlighted sentence
        :rtype: string
        """
        colored_text = ""
        init_pos = 0
        for ent in entities:
            if ent[1] > init_pos:
                colored_text += text[init_pos: ent[1]]
                colored_text += self.text_color(text[ent[1]: ent[2]]) + f"({ent[0]})"
                init_pos = ent[2]
            else:
                colored_text += self.text_color(text[ent[1]: ent[2]]) + f"({ent[0]})"
                init_pos = ent[2]
        
        return colored_text
    
    def get_entities(self, text: str):
        """
        Extracting entities from text with them position in text
        
        :param text: input sentence for preparing
        :type text: string
        :return: list with entities from text
        :rtype: list
        """
        assert len(text) > 0, text
        entities = self.token_pred_pipeline(text)
        concat_ent = self.concat_entities(entities)
        
        return concat_ent
    
    def show_ents_on_text(self, text: str):
        """
        Highlighting named entities in input text 
        
        :param text: input sentence for preparing
        :type text: string
        :return: Highlighting text
        :rtype: string
        """
        assert len(text) > 0, text
        entities = self.get_entities(text)
        
        return self.colored_text(text, entities)

In [12]:
# загрузим саму модель

extractor = Ner_Extractor(model_checkpoint = "surdan/LaBSE_ner_nerel")

In [13]:
# напишем небольшую функцию, которая вернет нам в удобном виде проанализированные предложения
# а также функцию для красивого отображения проанализированных предложений

def entities(seqs_example):
    l_entities = [extractor.get_entities(i) for i in seqs_example]
    print(len(l_entities), len(seqs_example))
    return np.array(l_entities)

def show_entities(seqs_example):
    show_entities_in_text = (extractor.show_ents_on_text(i) for i in seqs_example)
    for i in range(len(seqs_example)):
        print(next(show_entities_in_text, "End of generator"))
        print("-*-"*25)

In [16]:
# запустим эту функцию на ВСЕХ предложениях диалога. Так как в тестовом задании,
# когда упоминали нахождение названий компаний не конкретизировали делать ли это у менеджера или нет
l_entities = entities(df['text'].values)
l_entities

480 480


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


array([list([['CITY', 0, 4]]), list([['CITY', 0, 4], ['EVENT', 5, 17]]),
       list([['EVENT', 0, 11]]), list([['PERSON', 11, 19]]),
       list([['PERSON', 0, 3]]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([['NUMBER', 97, 98]]),
       list([]), list([]), list([['NUMBER', 45, 47]]), list([]), list([]),
       list([['PERSON', 0, 5]]), list([]),
       list([['NUMBER', 13, 14], ['DATE', 22, 40]]), list([]), list([]),
       list([['AGE', 87, 93]]),
       list([['DATE', 0, 1], ['DATE', 2, 7], ['DATE', 8, 9], ['NUMBER', 10, 11]]),
       list([['DATE', 27, 34]]), list([]), list([]), list([]), list([]),
       list([]), list([['PERSON', 0, 3]]), list([]),
       list([['AGE', 10, 12]]), list([]),
       list([['DISTRICT', 0, 3], ['EVENT', 4, 10]]), list([]), list([]),
       list([]), list([]), list([['DATE', 0, 7], ['AGE', 8, 12]]),
       list([]), list([]), list([]), list([]), list([]),
       list([['DATE', 29, 37]]), list([]), list

In [17]:
# для более удобной обработки тегов напишем вспомогательную функцию, которая в базе данных находит 
# все вхождения какого либо тега

def find_tag(df, entities, tag, indices=None):
    tags = []
    if indices == None:
        indices = range(len(entities))

    for i in indices:
        if len(entities[i]) > 0:
            for j in range(len(entities[i])):
                if entities[i][j][0] == tag:
                    tags.append((df['text'][i][entities[i][j][1]:entities[i][j][2]], i))
    return tags

In [18]:
# имена менеджера логичнее всего искать в предложениях, которые мы уже пометили как "представился"
# поэтому мы не будем запускать функцию для всей базы данных, а только для помеченных предложений

find_tag(df, l_entities, 'PERSON', name_indices)

[('ангелина', 3), ('ангелина', 111), ('ангелина', 167), ('максим', 251)]

In [19]:
# к сожалению, в клетке выше не отметился случай, с менеджером Анастасией на строчке 338
# давайте посмотрим какие теги отметились в данном предложении чтобы проанализировать на будущее

show_entities([df['text'][338]])

Да это анастасия(EVENT)
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-


In [20]:
# сверху мы видим, что "анастасия" была отмечена как событие. Однако же если написать то же предложение, но
# имя написать с заглавной буквы, то результат будет другим:

show_entities(['Да это Анастасия'])

Да это Анастасия(PERSON)
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-


In [21]:
# таким образом сеть работает, хорошо, но когда данные не учитывают регистр, это затрудняет делать предположения сети. 

# Перейдем к нахождению имен компаний в базе данных. В нашем случае достаточно запустить функцию на тег 'ORGANIZATION'
# так как не упоминалось что нужно найти названия компании только у менеджера, будем искать по всем предложения тег ORGANIZATION

find_tag(df, l_entities, 'ORGANIZATION')

[('техническому кабинету', 149), ('Диджитал', 177)]

In [22]:
# видим, что сеть, как минимум, не нашла упоминания менеджеров про китобизнес и диджитал бизнес, давайте попробуем проанализировать

print(df['text'][3])
show_entities([df['text'][3]])
print(df['text'][251])
show_entities([df['text'][251]])

Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
Меня зовут ангелина(PERSON)
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
Добрый меня максим зовут компания китобизнес удобно говорить
Добрый меня максим(PERSON)
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-


Ни диджитал бизнес ни китобизнес не отметились сетью в данном случае, однако мы помним, что Диджитал отметился как организация, когда стоял на первом месте предложения с большой буквой. Опять же подтверждается чувствительность к регистру. Тег "PERSON" или "ORGANIZATION" и вправду намного вероятнее если написан с большой буквы.

In [23]:
# в случае компании "китобизнес" также интересный момент, она правильно помечается, если не присутствуют 
# последующие слова, которые сбивают сеть. Видимо она разбивает предложение на n-граммы и в таком случае
# вероятности меняются 

show_entities(['Добрый меня максим зовут компания китобизнес удобно'])
show_entities(['Добрый меня максим зовут компания китобизнес'])

Добрый меня максим(PERSON)
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
Добрый меня максим(PERSON) зовут компания китобизнес(ORGANIZATION)
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-


In [24]:
# перейдем к последнему заданию и будем проверять поздоровался и попрощался ли менеджер с клиентом
# как и советовалось в задании, добавим дополнительные столбцы-флажки, которые будут отвечать за
# приветствие, представление и прощание. Матрица в таком случае получается немного разреженной к сожалению

df['greeted'] = False
df['introduced'] = False
df['said_goodbye'] = False

In [25]:
# так как мы уже нашли все необходимые индексы, то пометить их не составит труда:

df['greeted'][greeting_indices] = True
df['introduced'][name_indices] = True
df['said_goodbye'][bye_indices] = True

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [26]:
# напишем функцию, которая проверяет для каждого телефонного разговора попрощался ли и поздоровался ли менеджер
# если что-то не так, то функция выведет айди телефонного разговора

def find_bad_managers():
    wrong_manager_call_id = []
    for i in df['dlg_id'].unique():
        if df['greeted'][df['dlg_id'] == i].sum() == 0 or df['said_goodbye'][df['dlg_id'] == i].sum() == 0:
            wrong_manager_call_id.append(i)
    return wrong_manager_call_id

In [27]:
# вызовем функцию и найдем наконец-то нарушителей!

find_bad_managers()

[2, 4, 5]